In [ ]:
# @title Definicje Klas Warstw (`Layer`, `Activation`)
import numpy as np

class Layer:
    """Reprezentuje warstwę w pełni połączoną (liniową)."""
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) * 0.1
        self.bias = np.zeros((1, output_size))
        self.input = None
        self.weights_error = None
        self.bias_error = None

    def forward(self, input_data):
        self.input = input_data
        return self.input @ self.weights # + bias

    def backward(self, output_error):
        self.weights_error = self.input.T @ output_error
        # self.bias_error = np.sum(output_error, axis=0)
        input_error = output_error @ self.weights.T
        return input_error

    def adjust(self, learning_rate):
        self.weights -= learning_rate * self.weights_error
        # self.bias -= learning_rate * self.bias_error

class Activation:
    """Reprezentuje warstwę aktywacji."""
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
        self.input = None

    def forward(self, input_data):
        self.input = input_data
        return self.activation(self.input)

    def backward(self, output_error):
        return output_error * self.activation_prime(self.input)

    def adjust(self, learning_rate):
        pass

def relu(x):
    """Funkcja aktywacji ReLU."""
    return np.maximum(0, x)

def relu_prime(x):
    """Pochodna funkcji ReLU."""
    return (x > 0).astype(float)

print("Klasy Layer i Activation zdefiniowane.")

Klasy Layer i Activation zdefiniowane.


In [ ]:
# @title L3 - Zadanie 1: Rozbudowa sieci o funkcję aktywacji ReLU
# section: main_execution (Zadanie 3.4.1)
print("=" * 80)
print("ROZWIĄZANIE ZADANIA 3.4.1 (ReLU bez uczenia)")
print("=" * 80)

# Dane wejściowe i oczekiwane wyniki
X_data = np.array([
    [0.5, 0.75, 0.1], [0.1, 0.3, 0.7], [0.2, 0.1, 0.6], [0.8, 0.9, 0.2]
])
expected_outputs = {
    "Seria 1": np.array([0.376, 0.3765, 0.305]),
    "Seria 2": np.array([0.082, 0.133, 0.123]),
    "Seria 3": np.array([0.053, 0.067, 0.073]),
    "Seria 4": np.array([0.49, 0.465, 0.402])
}

# Wagi z instrukcji
Wh = np.array([
    [0.1, 0.1, -0.3], [0.1, 0.2, 0.0], [0.0, 0.7, 0.1],
    [0.2, 0.4, 0.0], [-0.3, 0.5, 0.1]
]).T
Wy = np.array([
    [0.7, 0.9, -0.4, 0.8, 0.1], [0.8, 0.5, 0.3, 0.1, 0.0],
    [-0.3, 0.9, 0.3, 0.1, -0.2]
]).T

# Budowa sieci
network = [
    Layer(3, 5),
    Activation(relu, relu_prime),
    Layer(5, 3)
]
network[0].weights = Wh
network[2].weights = Wy

# Predykcja i Weryfikacja
all_correct = True
np.set_printoptions(suppress=True, precision=8)

for i in range(len(X_data)):
    seria_name = f"Seria {i+1}"
    x = X_data[i].reshape(1, -1)

    # Podanie do przodu
    output = x
    for layer in network:
        output = layer.forward(output)

    output = output.flatten()

    print(f"\n--- {seria_name} ---")
    print(f"Otrzymany wynik:  {output}")
    print(f"Oczekiwany wynik: {expected_outputs[seria_name]}")

    if not np.allclose(output, expected_outputs[seria_name], atol=1e-4):
        all_correct = False

print("\n" + "="*80)
if all_correct:
    print("[ SUKCES ] Wszystkie wyniki są zgodne.")
else:
    print("[ BŁĄD ] Niektóre wyniki są niezgodne.")

ROZWIĄZANIE ZADANIA 3.4.1 (ReLU bez uczenia)

--- Seria 1 ---
Otrzymany wynik:  [0.376  0.3765 0.305 ]
Oczekiwany wynik: [0.376  0.3765 0.305 ]

--- Seria 2 ---
Otrzymany wynik:  [0.082 0.133 0.123]
Oczekiwany wynik: [0.082 0.133 0.123]

--- Seria 3 ---
Otrzymany wynik:  [0.053 0.067 0.073]
Oczekiwany wynik: [0.053 0.067 0.073]

--- Seria 4 ---
Otrzymany wynik:  [0.49  0.465 0.402]
Oczekiwany wynik: [0.49  0.465 0.402]

[ SUKCES ] Wszystkie wyniki są zgodne.


In [ ]:
# @title L3 - Zadanie 2: Implementacja pełnej propagacji wstecznej
# section: main_execution (Zadanie 3.4.2)
print("=" * 80)
print("ROZWIĄZANIE ZADANIA 3.4.2 (Uczenie z Backpropagation)")
print("=" * 80)

# Dane wejściowe i oczekiwane wyniki
X_train = np.array([
    [0.5, 0.75, 0.1], [0.1, 0.3, 0.7], [0.2, 0.1, 0.6], [0.8, 0.9, 0.2]
])
Y_train = np.array([
    [0.1, 1.0, 0.1], [0.5, 0.2, -0.5], [0.1, 0.3, 0.2], [0.7, 0.6, 0.2]
])

# Wagi początkowe z instrukcji
initial_Wh = np.array([
    [0.1, 0.1, -0.3], [0.1, 0.2, 0.0], [0.0, 0.7, 0.1],
    [0.2, 0.4, 0.0], [-0.3, 0.5, 0.1]
]).T
initial_Wy = np.array([
    [0.7, 0.9, -0.4, 0.8, 0.1], [0.8, 0.5, 0.3, 0.1, 0.0],
    [-0.3, 0.9, 0.3, 0.1, -0.2]
]).T

# Parametry uczenia
ALPHA = 0.01
EPOCHS = 50

# Oczekiwane wyniki do weryfikacji
expected_results_verification = {
    1: np.array([
        [0.376, 0.3765, 0.305],
        [0.0807193, 0.134082, 0.122503],
        [0.053154, 0.0670564, 0.07174],
        [0.490493, 0.470195, 0.398576]
    ]),
    50: np.array([
        [0.442756, 0.565494, 0.15731],
        [0.140418, 0.186105, 0.0638947],
        [0.105594, 0.0936588, 0.0450485],
        [0.584537, 0.711427, 0.216373]
    ])
}

# Budowa sieci
network = [
    Layer(3, 5),
    Activation(relu, relu_prime),
    Layer(5, 3)
]
network[0].weights = initial_Wh.copy()
network[2].weights = initial_Wy.copy()

# Pętla uczenia
history = {}
for epoch in range(1, EPOCHS + 1):
    epoch_outputs = []
    for x, y in zip(X_train, Y_train):
        output = x.reshape(1, -1)

        # Forward pass
        for layer in network:
            output = layer.forward(output)
        epoch_outputs.append(output.flatten())

        # Obliczanie błędu
        error = 2 * (output - y) / y.size

        # Backward pass
        for layer in reversed(network):
            error = layer.backward(error)

        # Aktualizacja wag
        for layer in network:
            layer.adjust(ALPHA)

    history[epoch] = np.array(epoch_outputs)
    print(f"Epoka {epoch} zakończona.")

# --- Weryfikacja wyników ---
np.set_printoptions(suppress=True, precision=8)
for epoch_to_check in [1, 50]:
    print("\n" + "=" * 80)
    print(f"WERYFIKACJA WYNIKÓW DLA EPOKI: {epoch_to_check}")
    print("=" * 80)

    actual_outputs = history[epoch_to_check]
    expected_outputs = expected_results_verification[epoch_to_check]

    for i in range(len(actual_outputs)):
        print(f"--- Seria {i+1} ---")
        print(f"  Otrzymany:  {actual_outputs[i]}")
        print(f"  Oczekiwany: {expected_outputs[i]}")

    if np.allclose(actual_outputs, expected_outputs, atol=1e-5):
        print("\n[ SUKCES ] Wyniki dla tej epoki są zgodne z oczekiwaniami.")
    else:
        print("\n[ BŁĄD ] Wyniki dla tej epoki są niezgodne.")

ROZWIĄZANIE ZADANIA 3.4.2 (Uczenie z Backpropagation)
Epoka 1 zakończona.
Epoka 2 zakończona.
Epoka 3 zakończona.
Epoka 4 zakończona.
Epoka 5 zakończona.
Epoka 6 zakończona.
Epoka 7 zakończona.
Epoka 8 zakończona.
Epoka 9 zakończona.
Epoka 10 zakończona.
Epoka 11 zakończona.
Epoka 12 zakończona.
Epoka 13 zakończona.
Epoka 14 zakończona.
Epoka 15 zakończona.
Epoka 16 zakończona.
Epoka 17 zakończona.
Epoka 18 zakończona.
Epoka 19 zakończona.
Epoka 20 zakończona.
Epoka 21 zakończona.
Epoka 22 zakończona.
Epoka 23 zakończona.
Epoka 24 zakończona.
Epoka 25 zakończona.
Epoka 26 zakończona.
Epoka 27 zakończona.
Epoka 28 zakończona.
Epoka 29 zakończona.
Epoka 30 zakończona.
Epoka 31 zakończona.
Epoka 32 zakończona.
Epoka 33 zakończona.
Epoka 34 zakończona.
Epoka 35 zakończona.
Epoka 36 zakończona.
Epoka 37 zakończona.
Epoka 38 zakończona.
Epoka 39 zakończona.
Epoka 40 zakończona.
Epoka 41 zakończona.
Epoka 42 zakończona.
Epoka 43 zakończona.
Epoka 44 zakończona.
Epoka 45 zakończona.
Epoka 46 z

In [ ]:
# @title L3 - Zadanie 3: Uczenie na danych MNIST
import urllib.request
import zipfile
import os
import struct
import time
import ssl

# section: data_handling_and_execution
def _download_data(url, save_path='.'):
    zip_path = os.path.join(save_path, 'MNIST_ORG.zip')
    data_dir = os.path.join(save_path, 'mnist_data')
    if not os.path.exists(data_dir):
        print("Pobieranie danych MNIST...")
        ssl_context = ssl._create_unverified_context()
        with urllib.request.urlopen(url, context=ssl_context) as response, open(zip_path, 'wb') as out_file:
            out_file.write(response.read())
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        os.remove(zip_path)
        print("Dane gotowe.")
    return data_dir

def _load_mnist(data_dir):
    def load_images(filename):
        with open(filename, 'rb') as f:
            _, num, rows, cols = struct.unpack('>IIII', f.read(16))
            return np.fromfile(f, dtype=np.uint8).reshape(num, rows * cols).astype(np.float32) / 255.0
    def load_labels(filename):
        with open(filename, 'rb') as f:
            _, num = struct.unpack('>II', f.read(8))
            labels = np.fromfile(f, dtype=np.uint8)
            one_hot = np.zeros((num, 10))
            one_hot[np.arange(num), labels] = 1
            return one_hot, labels

    X_train = load_images(os.path.join(data_dir, 'train-images.idx3-ubyte'))
    Y_train, y_train_labels = load_labels(os.path.join(data_dir, 'train-labels.idx1-ubyte'))
    X_test = load_images(os.path.join(data_dir, 't10k-images.idx3-ubyte'))
    _, y_test_labels = load_labels(os.path.join(data_dir, 't10k-labels.idx1-ubyte'))
    return X_train, Y_train, y_train_labels, X_test, y_test_labels

# Parametry Eksperymentu
# Możesz zmieniać te wartości, aby odtworzyć różne logi z instrukcji
ALPHA = 0.005
EPOCHS = 350
HIDDEN_SIZE = 100
TRAIN_SET_SIZE = 1000
TEST_SET_SIZE = 10000

# Wczytywanie danych
data_dir = _download_data('http://pduch.kis.p.lodz.pl/PSI/MNIST_ORG.zip')
X_train, Y_train, y_train_labels, X_test, y_test_labels = _load_mnist(data_dir)

# Budowa sieci
network = [
    Layer(784, HIDDEN_SIZE),
    Activation(relu, relu_prime),
    Layer(HIDDEN_SIZE, 10)
]

print(f"\nStart uczenia: Alpha: {ALPHA}, Train Size: {TRAIN_SET_SIZE}, Hidden: {HIDDEN_SIZE}")
start_time = time.time()

# Pętla uczenia
for epoch in range(EPOCHS):
    total_error = 0
    indices = np.arange(TRAIN_SET_SIZE)
    np.random.shuffle(indices)

    for i in indices:
        x = X_train[i].reshape(1, -1)
        y = Y_train[i].reshape(1, -1)

        # Forward pass
        output = x
        for layer in network:
            output = layer.forward(output)

        # Obliczanie błędu
        total_error += np.sum((y - output)**2)
        error = 2 * (output - y) / y.size

        # Backward pass
        for layer in reversed(network):
            error = layer.backward(error)

        # Aktualizacja wag
        for layer in network:
            layer.adjust(ALPHA)

    # Ewaluacja i logowanie co 10 epok
    if epoch % 10 == 0 or epoch == EPOCHS - 1:
        correct = 0
        for i in range(TEST_SET_SIZE):
            x_test = X_test[i].reshape(1, -1)
            output = x_test
            for layer in network:
                output = layer.forward(output)
            if np.argmax(output) == y_test_labels[i]:
                correct += 1
        accuracy = (correct / TEST_SET_SIZE) * 100
        print(f"Iter: {epoch} Error: {total_error/TRAIN_SET_SIZE:.6f} Test Accuracy: {accuracy:.2f}%")

print(f"Uczenie zakończone w {time.time() - start_time:.2f}s")


Start uczenia: Alpha: 0.005, Train Size: 1000, Hidden: 100
Iter: 0 Error: 1.595954 Test Accuracy: 40.65%
Iter: 10 Error: 0.411026 Test Accuracy: 76.70%
Iter: 20 Error: 0.315225 Test Accuracy: 80.31%
Iter: 30 Error: 0.263240 Test Accuracy: 81.94%
Iter: 40 Error: 0.229265 Test Accuracy: 83.15%
Iter: 50 Error: 0.203070 Test Accuracy: 83.64%
Iter: 60 Error: 0.182812 Test Accuracy: 84.34%
Iter: 70 Error: 0.166537 Test Accuracy: 84.62%
Iter: 80 Error: 0.152552 Test Accuracy: 84.64%
Iter: 90 Error: 0.140984 Test Accuracy: 85.21%
Iter: 100 Error: 0.130414 Test Accuracy: 85.49%
Iter: 110 Error: 0.121599 Test Accuracy: 85.57%
Iter: 120 Error: 0.113382 Test Accuracy: 85.70%
Iter: 130 Error: 0.106211 Test Accuracy: 85.83%
Iter: 140 Error: 0.099655 Test Accuracy: 85.90%
Iter: 150 Error: 0.093839 Test Accuracy: 85.99%
Iter: 160 Error: 0.088751 Test Accuracy: 86.00%
Iter: 170 Error: 0.083667 Test Accuracy: 86.00%
Iter: 180 Error: 0.079290 Test Accuracy: 86.01%
Iter: 190 Error: 0.075310 Test Accuracy

In [ ]:
# @title L3 - Zadanie 4: Uczenie Sieci Głębokiej do rozpoznawania kolorów
import urllib.request
import ssl
import time
import numpy as np

# --- Dane i funkcje pomocnicze ---
def load_color_data(url: str) -> tuple[np.ndarray, np.ndarray]:
    ssl_context = ssl._create_unverified_context()
    with urllib.request.urlopen(url, context=ssl_context) as response:
        data = response.read().decode('utf-8')
    X, Y = [], []
    for line in data.strip().split('\n'):
        parts = line.strip().split()
        if len(parts) == 4:
            r, g, b, color_id = map(float, parts)
            X.append([r, g, b])
            Y.append(int(color_id))
    return np.array(X), np.array(Y)

def labels_to_one_hot(labels: np.ndarray, num_classes: int) -> np.ndarray:
    one_hot = np.zeros((len(labels), num_classes))
    one_hot[np.arange(len(labels)), labels - 1] = 1.0
    return one_hot

# Parametry
HIDDEN_NEURONS_DEEP = 10
ALPHA_DEEP = 0.01
EPOCHS_DEEP = 100

# Wczytywanie danych
X_train, y_train_labels = load_color_data("http://pduch.iis.p.lodz.pl/PSI/training_colors.txt")
X_test, y_test_labels = load_color_data("http://pduch.iis.p.lodz.pl/PSI/test_colors.txt")
Y_train_one_hot = labels_to_one_hot(y_train_labels, 4)

# Budowa sieci
network_deep = [
    Layer(3, HIDDEN_NEURONS_DEEP),
    Activation(relu, relu_prime),
    Layer(HIDDEN_NEURONS_DEEP, 4)
]

print("="*80)
print(f"Start uczenia SIECI GŁĘBOKIEJ ({HIDDEN_NEURONS_DEEP} neuronów ukrytych)...")
print("="*80)
start_time = time.time()

# Pętla uczenia
achieved_100_at_epoch_deep = -1
for epoch in range(EPOCHS_DEEP):
    for x, y in zip(X_train, Y_train_one_hot):
        output = x.reshape(1, -1)
        for layer in network_deep: output = layer.forward(output)
        error = 2 * (output - y) / y.size
        for layer in reversed(network_deep): error = layer.backward(error)
        for layer in network_deep: layer.adjust(ALPHA_DEEP)

    # Ewaluacja
    correct = 0
    for x_t, y_t in zip(X_test, y_test_labels):
        output = x_t.reshape(1, -1)
        for layer in network_deep: output = layer.forward(output)
        if (np.argmax(output) + 1) == y_t: correct += 1
    accuracy = (correct / len(X_test)) * 100

    print(f"Epoka {epoch:3d}: Dokładność na teście = {accuracy:.2f}%")

    if accuracy == 100.0 and achieved_100_at_epoch_deep == -1:
        achieved_100_at_epoch_deep = epoch

# Zatrzymanie licznika czasu po pętli treningowej
total_time = time.time() - start_time

print(f"\nUczenie zakończone w {total_time:.2f}s")
if achieved_100_at_epoch_deep != -1:
    print(f"[ WYNIK ] Sieć głęboka osiągnęła 100% w epoce: {achieved_100_at_epoch_deep}")
else:
    print(f"[ WYNIK ] Sieć głęboka nie osiągnęła 100% w {EPOCHS_DEEP} epokach.")

Start uczenia SIECI GŁĘBOKIEJ (10 neuronów ukrytych)...
Epoka   0: Dokładność na teście = 25.38%
Epoka   1: Dokładność na teście = 25.38%
Epoka   2: Dokładność na teście = 36.92%
Epoka   3: Dokładność na teście = 52.31%
Epoka   4: Dokładność na teście = 52.31%
Epoka   5: Dokładność na teście = 52.31%
Epoka   6: Dokładność na teście = 52.31%
Epoka   7: Dokładność na teście = 52.31%
Epoka   8: Dokładność na teście = 51.54%
Epoka   9: Dokładność na teście = 50.77%
Epoka  10: Dokładność na teście = 49.23%
Epoka  11: Dokładność na teście = 46.92%
Epoka  12: Dokładność na teście = 46.92%
Epoka  13: Dokładność na teście = 46.92%
Epoka  14: Dokładność na teście = 47.69%
Epoka  15: Dokładność na teście = 52.31%
Epoka  16: Dokładność na teście = 62.31%
Epoka  17: Dokładność na teście = 71.54%
Epoka  18: Dokładność na teście = 75.38%
Epoka  19: Dokładność na teście = 76.92%
Epoka  20: Dokładność na teście = 76.92%
Epoka  21: Dokładność na teście = 77.69%
Epoka  22: Dokładność na teście = 78.46%
E

In [ ]:
# @title L2 - Zadanie 3: Rozpoznawanie kolorów RGB

import numpy as np
import urllib.request
import time  # Import modułu time

class ColorRecognitionNetwork:
    def __init__(self, weights: np.ndarray, alpha: float = 0.01):
        self.weights = weights.copy()
        self.alpha = alpha

    def forward(self, x: np.ndarray) -> np.ndarray:
        x_col = x.reshape(-1, 1)
        output = self.weights @ x_col
        return output.flatten()

    def predict_color(self, x: np.ndarray) -> int:
        output = self.forward(x)
        return np.argmax(output) + 1

    def calculate_error(self, prediction: np.ndarray, goal: np.ndarray) -> float:
        N = len(prediction)
        return (1 / N) * np.sum((prediction - goal) ** 2)

    def calculate_delta(self, prediction: np.ndarray, goal: np.ndarray, x: np.ndarray) -> np.ndarray:
        N = len(prediction)
        diff = prediction - goal
        return (2 / N) * np.outer(diff, x)

    def update_weights(self, delta: np.ndarray) -> None:
        self.weights = self.weights - self.alpha * delta

    def train_epoch(self, X: np.ndarray, Y: np.ndarray) -> float:
        total_error = 0.0

        for x, y in zip(X, Y):
            prediction = self.forward(x)
            total_error += self.calculate_error(prediction, y)
            delta = self.calculate_delta(prediction, y, x)
            self.update_weights(delta)

        return total_error

    def train(self, X: np.ndarray, Y: np.ndarray, X_train_eval: np.ndarray, Y_train_labels_eval: np.ndarray, epochs: int) -> tuple[int, float]:
        """
        Metoda trenująca sieć, zwracająca numer epoki z 100% dokładnością i całkowity czas.
        """
        start_time = time.time()
        epoch_100_percent = -1

        for epoch in range(epochs):
            total_error = self.train_epoch(X, Y)

            # Sprawdzanie dokładności i zapisywanie numeru epoki, jeśli osiągnięto 100%
            if epoch_100_percent == -1:
                current_accuracy = self.evaluate(X_train_eval, Y_train_labels_eval)
                if current_accuracy == 100.0:
                    epoch_100_percent = epoch

            if epoch % 10 == 0 or epoch == epochs - 1:
                print(f"Epoka {epoch:4d}: Błąd = {total_error:.6f}")

        end_time = time.time()
        total_training_time = end_time - start_time
        return epoch_100_percent, total_training_time

    def evaluate(self, X_test: np.ndarray, Y_test_labels: np.ndarray) -> float:
        correct = 0
        for x, true_label in zip(X_test, Y_test_labels):
            if self.predict_color(x) == true_label:
                correct += 1
        return (correct / len(X_test)) * 100


def load_color_data(url: str) -> tuple[np.ndarray, np.ndarray]:
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf-8')

    X, Y = [], []
    for line in data.strip().split('\n'):
        parts = line.strip().split()
        if len(parts) == 4:
            r, g, b, color_id = map(float, parts)
            X.append([r, g, b])
            Y.append(int(color_id))

    return np.array(X), np.array(Y)


def labels_to_one_hot(labels: np.ndarray) -> np.ndarray:
    one_hot = np.zeros((len(labels), 4))
    for i, label in enumerate(labels):
        one_hot[i, label - 1] = 1.0
    return one_hot


# ==============================================================================
# ZADANIE 3: Rozpoznawanie kolorów RGB
# ==============================================================================
print("=" * 80)
print("ZADANIE 3: Rozpoznawanie kolorów RGB")
print("=" * 80)

# Pobieranie danych
X_train, Y_train_labels = load_color_data("https://pduch.iis.p.lodz.pl/PSI/training_colors.txt")
X_test, Y_test_labels = load_color_data("http://pduch.iis.p.lodz.pl/PSI/test_colors.txt")

print(f"Dane treningowe: {len(X_train)} próbek")
print(f"Dane testowe: {len(X_test)} próbek\n")

# Konwersja etykiet na one-hot
Y_train_one_hot = labels_to_one_hot(Y_train_labels)

# Inicjalizacja i uczenie sieci
initial_weights = np.random.uniform(-0.5, 0.5, size=(4, 3))
network = ColorRecognitionNetwork(weights=initial_weights, alpha=0.01)

# Trening z pomiarem czasu i śledzeniem epoki dla 100% dokładności
epoch_100, total_time = network.train(X_train, Y_train_one_hot, X_train, Y_train_labels, epochs=100)

# Wyniki
print("\n" + "=" * 80)
print("WYNIKI")
print("=" * 80)
train_accuracy = network.evaluate(X_train, Y_train_labels)
test_accuracy = network.evaluate(X_test, Y_test_labels)

print(f"Dokładność treningowa: {train_accuracy:.2f}%")
print(f"Dokładność testowa:    {test_accuracy:.2f}%")

# Wyświetlanie dodatkowych informacji
print(f"\nCałkowity czas uczenia: {total_time:.4f} sekundy")
if epoch_100 != -1:
    print(f"100% dokładności na zbiorze treningowym osiągnięto w epoce: {epoch_100}")
else:
    print("Nie osiągnięto 100% dokładności na zbiorze treningowym w trakcie uczenia.")


if test_accuracy == 100.0:
    print("\nSUKCES! Sieć osiągnęła 100% dokładności na zbiorze testowym!")

# Test
print("\n" + "=" * 80)
print("TEST")
print("=" * 80)
test_colors = [
    ([0.9, 0.1, 0.1], "Czerwony", 1),
    ([0.1, 0.9, 0.1], "Zielony", 2),
    ([0.1, 0.1, 0.9], "Niebieski", 3),
    ([0.9, 0.9, 0.1], "Żółty", 4)
]

color_names = {1: "Czerwony", 2: "Zielony", 3: "Niebieski", 4: "Żółty"}

for rgb, expected_name, expected_id in test_colors:
    predicted_id = network.predict_color(np.array(rgb))
    status = "✓" if predicted_id == expected_id else "✗"
    print(f"{status} RGB: {rgb} -> {color_names[predicted_id]} (oczekiwane: {expected_name})")

ZADANIE 3: Rozpoznawanie kolorów RGB
Dane treningowe: 109 próbek
Dane testowe: 130 próbek

Epoka    0: Błąd = 36.424794
Epoka   10: Błąd = 10.188887
Epoka   20: Błąd = 8.028996
Epoka   30: Błąd = 7.594466
Epoka   40: Błąd = 7.505781
Epoka   50: Błąd = 7.487513
Epoka   60: Błąd = 7.483731
Epoka   70: Błąd = 7.482949
Epoka   80: Błąd = 7.482790
Epoka   90: Błąd = 7.482759
Epoka   99: Błąd = 7.482754

WYNIKI
Dokładność treningowa: 100.00%
Dokładność testowa:    100.00%

Całkowity czas uczenia: 0.2103 sekundy
100% dokładności na zbiorze treningowym osiągnięto w epoce: 12

SUKCES! Sieć osiągnęła 100% dokładności na zbiorze testowym!

TEST
✓ RGB: [0.9, 0.1, 0.1] -> Czerwony (oczekiwane: Czerwony)
✓ RGB: [0.1, 0.9, 0.1] -> Zielony (oczekiwane: Zielony)
✓ RGB: [0.1, 0.1, 0.9] -> Niebieski (oczekiwane: Niebieski)
✓ RGB: [0.9, 0.9, 0.1] -> Żółty (oczekiwane: Żółty)
